In [1]:
import numpy as np
from ecog_band.utils import *
import pandas as pd
import numpy as np
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
from matplotlib.cm import ScalarMappable
from ecog_band.datasetAllband import SVMDataset
from ecog_band.models import SVMBinClassifier
import os
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from ecog_band.utils import *
from ecog_band.solver import Nfold_solver
import pandas as pd
# from ecog_band.datasetExcludeBand import CustomDatasetExcband
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix

HS = 69
freq = 500

rootPath = '/root/pp/covert-reading/Ecog_pretrain/accuracy_results_svm_avgfreq'
path = "/public/DATA/overt_reading/"

band_list = get_all_band()
n_freq_bands = len(band_list)

/root/pp/covert-reading/Ecog_pretrain/ecog_band/solver.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm  # may raise warning about Jupyter


In [2]:
# 所有电极在不同频段的acc
idx_elec = list(range(1, 256))
for elec in idx_elec:
    path_elec = f'/public/DATA/overt_reading/dataset_/HS{HS}/{freq}/{elec}'
    num_samples = len(os.listdir(path_elec))
    y_save_path = f'/root/pp/covert-reading/Ecog_pretrain/accuracy_results_svm_avgfreq/HS{HS}/{freq}/{elec}'
    fig_save_path = f'/root/pp/covert-reading/Ecog_pretrain/accuracy_results_svm_avgfreq/HS{HS}/{freq}/{elec}/figs'
    os.makedirs(y_save_path, exist_ok=True)
    os.makedirs(fig_save_path, exist_ok=True)
    for band in band_list:
        print(f'Calculate for HS{HS} elec{elec} band{band}')
        data_loader = SVMDataset(HS, path_elec, freq, elec, num_samples, band, exclude=False, avg='avgFreq')
        data, labels = data_loader.get_data_labels()
        print(f'data_shape: {data.shape}')
        X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=1/6, random_state=42)
        # svm = DecisionTreeBinClassifier()
        svm = SVMBinClassifier()
        svm.train(x_train=X_train, y_train=y_train)

        y_pred = svm.evaluate(X_test=X_test, y_test=y_test)
        band_acc = accuracy_score(y_test, y_pred)
        print(f"Accuracy on test set band_{band}: {band_acc}")

        np.save(os.path.join(y_save_path, f'{band}_y_pred.npy'), y_pred)
        np.save(os.path.join(y_save_path, f'{band}_y_true.npy'), y_test)
        

Calculate for HS69 elec1 bandelse1
data_shape: (720, 375)
Accuracy on test set band_else1: 0.6333333333333333
Calculate for HS69 elec1 banddelta
data_shape: (720, 375)
Accuracy on test set band_delta: 0.6666666666666666
Calculate for HS69 elec1 bandtheta
data_shape: (720, 375)
Accuracy on test set band_theta: 0.625
Calculate for HS69 elec1 bandalpha
data_shape: (720, 375)
Accuracy on test set band_alpha: 0.6666666666666666
Calculate for HS69 elec1 bandbeta
data_shape: (720, 375)
Accuracy on test set band_beta: 0.7416666666666667
Calculate for HS69 elec1 bandgamma
data_shape: (720, 375)
Accuracy on test set band_gamma: 0.7
Calculate for HS69 elec1 bandhigh gamma
data_shape: (720, 375)
Accuracy on test set band_high gamma: 0.8583333333333333
Calculate for HS69 elec1 bandelse2
data_shape: (720, 375)
Accuracy on test set band_else2: 0.65
Calculate for HS69 elec2 bandelse1
data_shape: (720, 375)
Accuracy on test set band_else1: 0.975
Calculate for HS69 elec2 banddelta
data_shape: (720, 375)

In [ ]:
# 所有电极在不同频段的acc
acc_dic = cal_all_elec_acc(rootPath, HS, freq)

# 数据加载
subject_str = f"HS{HS}"
xy = scio.loadmat(path + f"/elecs/All_elecs/{subject_str}_warped_elec_pos2D.mat")['elecmatrix']
anatomy = scio.loadmat(path + f"/elecs/All_elecs/{subject_str}_elecs_all_warped.mat")['anatomy']
# sig_elecs_HS = np.load(path + f"/elecs/sig_elecs/{subject_str}_sig_elecs5.npy", allow_pickle=True).item()

# 颜色映射
color_maps = [cm.get_cmap('viridis', n_freq_bands) for _ in range(n_freq_bands)]  # 颜色映射
norms = [plt.Normalize(0, 1) for _ in range(n_freq_bands)]  # 频段准确率的规范化

# 加载大脑模板图像
img = mpimg.imread(path + "/elecs/Brain2D/MNI152_brain2D.png")

fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(img)

# 绘制每个频段的电极
for band in range(n_freq_bands):
    x_coords, y_coords, accuracy_values = [], [], []
    
    for elec in idx_elec:
        x_coords.append(xy[elec][0])
        y_coords.append(xy[elec][1])
        
        # 获取该频段的准确率
        accuracy = acc_dic[elec][band]
        accuracy_values.append(accuracy)
    
    # 绘制电极，使用颜色映射
    sc = ax.scatter(x_coords, y_coords, c=accuracy_values, cmap=color_maps[band], norm=norms[band], alpha=0.5, s=100, label=f'Band {band+1}')

# 添加颜色条
cbar = plt.colorbar(ScalarMappable(norm=norms[0], cmap=color_maps[0]), ax=ax)
cbar.set_label('Accuracy', size=15)

# 添加图例
ax.legend(title="Frequency Bands", bbox_to_anchor=(1.05, 1), loc='upper left')

# 额外设置
ax.grid('off')
ax.axis('off')
ax.set_xticks([])
ax.set_yticks([])
ax.set_title(f"Electrodes Distribution with Frequency Band Accuracy for HS{HS}", size=25)

plt.savefig(f"/root/pp/covert-reading/Ecog_pretrain/figures/fig_acc_distri_HS{HS}_freq{freq}.pdf", dpi=1200)
plt.show()